<a href="https://colab.research.google.com/github/maticvl/dataHacker/blob/master/CNN/CNN_DeepDream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import inception_v3
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras import backend as K
from scipy.ndimage import zoom
from scipy.ndimage.filters import gaussian_filter
from imageio import imwrite

Using TensorFlow backend.


In [0]:
K.image_data_format()

'channels_last'

In [0]:
model = inception_v3.InceptionV3(weights = 'imagenet', include_top = False )

In [0]:
layer_contributions = { 'mixed3':3,'mixed4':2, 'mixed5':1.5, 'mixed9':2, 'conv2d_94':2}

#layer_contributions = { 'mixed3':3 , 'mixed9':2 }

#layer_contributions = {'conv2d_16':1, 'mixed3':3, 'mixed5':1.5}

#layer_contributions = { 'mixed3':3,'mixed4':2, 'conv2d_94':2}

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
layer_dict = dict([(layer.name, layer) for layer in model.layers ])

In [0]:
layer_dict

{'input_1': <keras.engine.input_layer.InputLayer at 0x22e85b90128>,
 'conv2d_1': <keras.layers.convolutional.Conv2D at 0x22e85c2df98>,
 'batch_normalization_1': <keras.layers.normalization.BatchNormalization at 0x22e85c2def0>,
 'activation_1': <keras.layers.core.Activation at 0x22e85c2ddd8>,
 'conv2d_2': <keras.layers.convolutional.Conv2D at 0x22efb2d81d0>,
 'batch_normalization_2': <keras.layers.normalization.BatchNormalization at 0x22e85c9be80>,
 'activation_2': <keras.layers.core.Activation at 0x22e85c9b630>,
 'conv2d_3': <keras.layers.convolutional.Conv2D at 0x22e85d4ab00>,
 'batch_normalization_3': <keras.layers.normalization.BatchNormalization at 0x22e85e28a20>,
 'activation_3': <keras.layers.core.Activation at 0x22e85da8dd8>,
 'max_pooling2d_1': <keras.layers.pooling.MaxPooling2D at 0x22e85e4fd30>,
 'conv2d_4': <keras.layers.convolutional.Conv2D at 0x22e85e7b710>,
 'batch_normalization_4': <keras.layers.normalization.BatchNormalization at 0x22e85e9ce10>,
 'activation_4': <keras.

In [0]:
# loss will be changed so, we declare it as a variable
loss = K.variable(0.0)

In [0]:
for layer_name in layer_contributions:
    # coefficients quantifies how much eac layer's activations contibutes to the loss that we want to maximize
    coefficient = layer_contributions[layer_name]
    # the output of a layer
    activation = layer_dict[layer_name].output
    scaling_factor = K.prod(K.cast(K.shape(activation), 'float32'))
    # loss is sum of losses in all layers
    loss = loss + coefficient*K.sum(K.square(activation))/scaling_factor

In [0]:
# The tensor dream holds the generated image
deep_dream = model.input

In [0]:
# we calculate the gradient of the input image wrt our loss function
gradients = K.gradients(loss, deep_dream)[0]
# gradient normalizaation - turns out that this is important step
gradients /= K.maximum(K.mean(K.abs(gradients)), 0.0000001)
outputs = [loss, gradients]

loss_and_grads = K.function([deep_dream], [loss, gradients])

In [0]:
def gradient_ascent(x, iterations, step, max_loss = None):
    loss_value, grads_value = 0,0
    for i in range(iterations):
        loss_value, grads_value = loss_and_grads([x])
        # if our loss function gets to the max value then we should break
        # deep dreams won't look pretty if we procede in the following steps
        if max_loss is not None and loss_value > max_loss:
            break
        print('Loss value at iteration ', i, ':', loss_value)
    
        # we update the input image
        # this is the crucial step for making the deep dreams
        x += step*grads_value
    return x

In [0]:
def preprocess(image_path):
    # load the image in the PIL format
    img = load_img(image_path)
    # make a NumPy array of it
    img = img_to_array(img)
    # make a 4D tennsor of it
    img = np.expand_dims(img, axis = 0)
    # preprocessing steps for feeding the image through the inception_v3 
    img = inception_v3.preprocess_input(img)
    return img

In [0]:
def deprocess(x):
    # making an image out of tensor
    x = x.reshape(x.shape[1], x.shape[2], 3) 
    x/=2
    x+=0.5
    x*=255.
    image = np.clip(x, 0,255).astype('uint8')
    return image

In [0]:
def resize_img(x, size):
    img = x.copy()
    factors = (1, float(size[0]/img.shape[1]),float(size[1]/img.shape[2]), 1)
    return zoom(img, factors, order = 1)  

In [0]:
def save_img(img, fname):
    pil_img = deprocess(np.copy(img))
    imwrite(fname, pil_img)

In [0]:
cfg = {"image_path" : 'got.jpg',
       "l_rate" : 0.01,
       "num_octave" : 3,
       "octave_scale" : 1.4,
       "iterations" : 30,
       "max_loss" : 50.,
    
}

In [0]:
def deep_dream(image_path, l_rate, num_octave, octave_scale, iterations, max_loss):
    
    img_preprocessed = preprocess(image_path)
    # processed image has the batch dimension
    original_shape = img_preprocessed.shape[1:3]
    successive_shapes = [original_shape]
    
    original_img = img_preprocessed.copy()
    shrunck_original_img = resize_img(original_img, successive_shapes[0]) # we take the smallest shape
    
    for i in range(0, num_octave):
        shape = tuple([int(dim/(octave_scale**i)) for dim in original_shape] )
        successive_shapes.append(shape)
    successive_shapes = successive_shapes[::-1]
    print('Shapes of original image, that we will procss: ', successive_shapes)
    
    for shape in successive_shapes:
        print('Preprocessing image shape: ', shape)
        img = resize_img(img_preprocessed, shape)
        # appying gradient ascent for each scale
        img = gradient_ascent(img, iterations = iterations, step = l_rate, max_loss = max_loss)
        upscaled_shrunk_original_img = resize_img(shrunck_original_img, shape)
        same_size_original = resize_img(original_img, shape)
        # calculating the lost detail
        lost_detail = same_size_original - upscaled_shrunk_original_img
        img += lost_detail
        # preparing for the next iteration 
        shrunk_original_img = resize_img(original_img, shape)
        name = ''.join(list(image_path)[:-4])+'_dream_at_shape'+str(shape)+'.jpg'
        save_img(img, fname = name)
    
    #saving final image
    f_name = ''.join(list(image_path)[:-4])+'_final_dream.jpg'
    save_img(img, fname = f_name)

In [0]:
deep_dream(**cfg)

Shapes of original image, that we will procss:  [(704, 1056), (985, 1478), (1380, 2070), (1380, 2070)]
Preprocessing image shape:  (704, 1056)
Loss value at iteration  0 : 2.5999277
Loss value at iteration  1 : 3.162093
Loss value at iteration  2 : 3.9012845
Loss value at iteration  3 : 4.699877
Loss value at iteration  4 : 5.579523
Loss value at iteration  5 : 6.49706
Loss value at iteration  6 : 7.4140587
Loss value at iteration  7 : 8.304545
Loss value at iteration  8 : 9.163407
Loss value at iteration  9 : 10.009405
Loss value at iteration  10 : 10.804069
Loss value at iteration  11 : 11.58229
Loss value at iteration  12 : 12.326757
Loss value at iteration  13 : 13.031355
Loss value at iteration  14 : 13.724984
Loss value at iteration  15 : 14.383502
Loss value at iteration  16 : 15.015279
Loss value at iteration  17 : 15.65203
Loss value at iteration  18 : 16.236258
Loss value at iteration  19 : 16.825583
Loss value at iteration  20 : 17.388685
Loss value at iteration  21 : 17.951

C:\Users\Andjelka\Anaconda3\envs\tensorflow sessions\lib\site-packages\scipy\ndimage\interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Loss value at iteration  0 : 2.313983
Loss value at iteration  1 : 2.7449858
Loss value at iteration  2 : 3.3896003
Loss value at iteration  3 : 4.143696
Loss value at iteration  4 : 4.9998417
Loss value at iteration  5 : 5.944893
Loss value at iteration  6 : 6.916074
Loss value at iteration  7 : 7.878686
Loss value at iteration  8 : 8.824211
Loss value at iteration  9 : 9.722062
Loss value at iteration  10 : 10.616051
Loss value at iteration  11 : 11.460037
Loss value at iteration  12 : 12.273386
Loss value at iteration  13 : 13.074901
Loss value at iteration  14 : 13.822903
Loss value at iteration  15 : 14.564393
Loss value at iteration  16 : 15.27522
Loss value at iteration  17 : 15.972214
Loss value at iteration  18 : 16.658207
Loss value at iteration  19 : 17.314932
Loss value at iteration  20 : 17.953707
Loss value at iteration  21 : 18.584803
Loss value at iteration  22 : 19.18418
Loss value at iteration  23 : 19.775816
Loss value at iteration  24 : 20.362616
Loss value at itera